# Combined movies
Craig Lage - 10-Jun-24 \
The goal of this is to create a combined movie of the Simonyi telescope system, including the TMA, M1M3, M2, both hexapods, and the camera rotator.  Also will highlight when exposures are being taken.

In [ ]:
import sys, time, os, yaml, warnings
import shlex, subprocess
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.patches import Polygon, Circle, Rectangle
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from scipy.ndimage import rotate
from astropy.time import Time, TimeDelta
from lsst.ts.xml.tables.m1m3 import FATable
from lsst_efd_client import EfdClient
import logging
logging.basicConfig(level='INFO')
mlogger = logging.getLogger('matplotlib')
mlogger.setLevel(logging.ERROR)

## Set up the necessary subroutines

In [ ]:
def heatMapZM1M3(df, ax, FATable, time, zmin, zmax, AOS=True):
    ax.set_xlabel("X position (m)")
    ax.set_yticklabels([])
    ax.set_xlim(-5.0, 5.0)
    ax.set_ylim(-5.0,5.0)
    ax.set_xticks([-4,-2,0,2,4])
    ax.set_yticks([-4,-2,0,2,4])
    if AOS:
        ax.set_title("M1M3 AOS forces (N)", fontsize=12)
    else:
        ax.set_title("M1M3 Z forces (N)", fontsize=12)
    if len(df) == 0:
        ax.text(-2, 0, "Not Available")
        return
    if AOS:
        # Since these are events, not telemetry, we want
        # the last index before the given time
        test = df[df.index < time]
        if len(test) == 0:
            return
        else:
            index = df.index.get_loc(test.index[-1])
    else:
        index = df.index.searchsorted(time)
    types = [['SAA','NA', 'o', 'Z'], ['DAA','Y_PLUS', '^', 'Y_PLUS'], ['DAA','Y_MINUS', 'v', 'Y_MINUS'], \
             ['DAA','X_PLUS', '>', 'X_PLUS'], ['DAA','X_MINUS', '<', 'X_MINUS']]

    for [type, orient, marker, label] in types:
        xs = []
        ys = []
        zs = []
        for i in range(len(FATable)):
            x = FATable[i].x_position
            y = FATable[i].y_position
            if FATable[i].actuator_type.name == type and FATable[i].orientation.name == orient:
                xs.append(x)
                ys.append(y)
                if AOS:
                    name=f"zForces{i}"
                else:
                    name=f"zForce{i}"
                zs.append(df.iloc[index][name])
                #print(AOS, i, df.iloc[index][name])
        if AOS:
            im = ax.scatter(xs, ys, marker=marker, c=zs, cmap='RdBu_r', \
                        norm=colors.SymLogNorm(linthresh=zmax/100.0, vmin=zmin, vmax=zmax), \
                         s=50, label=label)
        else:    
            im = ax.scatter(xs, ys, marker=marker, c=zs, cmap='RdBu_r', \
                        vmin=zmin, vmax=zmax, s=50, label=label)
    axins = inset_axes(
        ax,
        width="5%",  # width: 50% of parent_bbox width
        height="100%",  # height: 5% of parent_bbox height
        loc='right',
        borderpad=0.0 # Add some padding from the border of the parent Axes
    )
    fig.colorbar(im, cax=axins, orientation="vertical")
    if AOS:
        axins.yaxis.set_ticks_position('left')
        axins.yaxis.set_label_position('left')
        axins.set_yticks([-100, 0, 100])
    else:
        axins.yaxis.set_ticks_position('right')
        axins.yaxis.set_label_position('right')
        axins.set_yticks([zmin, 0, zmax])
        
    return
    
def lateralForcesM1M3(df, ax, FATable, time, forceMax):
    ax.set_xlabel("X position (m)")
    ax.set_ylabel("Y position (m)")
    ax.set_title("M1M3 XY forces (N)", fontsize=12)
    ax.set_xlim(-5.0,5.0)
    ax.set_ylim(-5.0,5.0)
    ax.set_xticks([-4,-2,0,2,4])
    ax.set_yticks([-4,-2,0,2,4])
    if len(df) == 0:
        ax.text(-2, 0, "Not Available")
        return
    index = df.index.searchsorted(time)
    types = [['DAA','Y_PLUS', '^', 'Y_PLUS','g'], ['DAA','Y_MINUS', 'v', 'Y_MINUS', 'cyan'], \
             ['DAA','X_PLUS', '>', 'X_PLUS', 'r'], ['DAA','X_MINUS', '<', 'X_MINUS', 'r']]
    for [type, orient, marker, label, color] in types:
        xs = []
        ys = []
        arrowXs = []
        arrowYs = []
        for i in range(len(FATable)):
            x = FATable[i].x_position
            y = FATable[i].y_position
            if FATable[i].actuator_type.name == type and FATable[i].orientation.name == orient:
                xs.append(x)
                ys.append(y)
                if orient == 'X_PLUS':
                    name = f"xForce{FATable[i].x_index}"
                    arrowXs.append((df.iloc[index][name] - df.iloc[0][name]) / forceMax)
                    arrowYs.append(0.0)
                if orient == 'X_MINUS':
                    name = f"xForce{FATable[i].x_index}"
                    arrowXs.append(-(df.iloc[index][name] - df.iloc[0][name]) / forceMax)
                    arrowYs.append(0.0)
                if orient == 'Y_PLUS':
                    name = f"yForce{FATable[i].y_index}"
                    arrowXs.append(0.0)
                    arrowYs.append((df.iloc[index][name] - df.iloc[0][name]) / forceMax)
                if orient == 'Y_MINUS':
                    name = f"yForce{FATable[i].y_index}"
                    arrowXs.append(0.0)
                    arrowYs.append(-(df.iloc[index][name] - df.iloc[0][name]) / forceMax)
            else:
                continue
        ax.scatter(xs, ys, marker=marker, color=color, s=50, label=label)
        for ii in range(len(xs)):
            ax.arrow(xs[ii], ys[ii], arrowXs[ii], arrowYs[ii], color=color)

    ax.plot([-4.0,-3.0], [-3.7,-3.7], color='g')
    ax.text(-4.0, -4.3, f"{forceMax} N")
    return

from mpl_toolkits.axes_grid1.inset_locator import inset_axes

def plotAxialForcesM2(df, yaml_file, ax, zmin, zmax, time, AOS=True):
    # Get the data from the yaml file
    scale = 2.5
    ax.set_xlim(-scale, scale)
    ax.set_ylim(-scale, scale)
    ax.set_xlabel("Y position (m)")
    if AOS:
        ax.set_ylabel("Y position (m)")
        ax.set_title("M2 AOS forces (N)", fontsize=12)

    if len(df) == 0:
        ax.text(-1, 0, "Not Available")
        return
    
    if AOS:
        # Since these are events, not telemetry, we want
        # the last index before the given time
        test = df[df.index < time]
        if len(test) == 0:
            return
        else:
            index = df.index.get_loc(test.index[-1])
    else:
        index = df.index.searchsorted(time)
        ax.set_yticklabels([])
        ax.set_title("M2 Z forces (N)", fontsize=12)


    with open(yaml_file, 'r') as stream:
        locations = yaml.safe_load(stream)
    axials = np.array(locations['locAct_axial'])
    xs = axials[:,0]
    ys = axials[:,1]
    zs = []
    for i in range(len(xs)):
        if AOS:
            name=f"axial{i}"
        else:
            name=f"measured{i}"
        force = df.iloc[index][name]
        #print(time, f"axial{i}" , force)       
        zs.append(force)

    if AOS:
        im = ax.scatter(xs, ys, marker='o', c=zs, cmap='RdBu_r', \
                        norm=colors.SymLogNorm(linthresh=zmax/100.0, vmin=zmin, vmax=zmax), \
                         s=80, label="Axial")
    else:
        im = ax.scatter(ys, xs, marker='o', c=zs, cmap='RdBu_r', vmin=zmin, vmax=zmax,\
                        s=80, label="Axial")
    axins = inset_axes(
        ax,
        width="5%",  # width: 50% of parent_bbox width
        height="100%",  # height: 5% of parent_bbox height
        loc='right',
        borderpad=0.0 # Add some padding from the border of the parent Axes
    )
    fig.colorbar(im, cax=axins, orientation="vertical")

    if AOS:
        axins.yaxis.set_ticks_position('left')
        axins.yaxis.set_label_position('left')
        axins.set_yticks([-10, 0, 10])
    else:
        axins.yaxis.set_ticks_position('right')
        axins.yaxis.set_label_position('right')
        axins.set_yticks([zmin, 0, zmax])

    return

def plotTangentForcesM2(df, yaml_file, ax, zmax, time):
    scale = 2.5
    ax.set_xlim(-scale, scale)
    ax.set_ylim(-scale, scale)
    if len(df) == 0:
        ax.text(-1, 0, "Not Available")
        return
    index = df.index.searchsorted(time)
    # Get the data from the yaml file
    with open(yaml_file, 'r') as stream:
        locations = yaml.safe_load(stream)
    # Now plot tangential actuator locations
    Rtan = locations['radiusActTangent'] # Radius in meters
    thetas = locations['locAct_tangent']
    width = 0.2
    height = 0.9
    poly = [[-height/2.0, 0],[height/2.0, 0], \
                     [height/2.0, width], [-height/2.0, width]]
    
    for i, theta in enumerate(thetas):
        theta *= np.pi / 180.0
        rot_matrix = np.array([[np.cos(theta), -np.sin(theta)],
                       [np.sin(theta), np.cos(theta)]]).transpose()
        rot_poly = []
        for point in poly:
            rot_poly.append(np.dot(rot_matrix, point))
        rot_poly = np.array(rot_poly)
        x = Rtan * np.cos(theta)
        y = Rtan * np.sin(theta)
        xy = np.array([y,x]) + rot_poly

        if i == 0:
            ax.add_patch(Polygon(
                xy= xy, linewidth=1, color='coral', \
                fill=True, label='Tangent'))
        else:
            ax.add_patch(Polygon(
                xy=xy, linewidth=1, color='coral', fill=True))
        name=f"measured{i}"
        force =  - df.iloc[index][name]
        length = 0.5 * force / zmax
        dx = length * np.sin(theta)
        dy =  - length * np.cos(theta)
        box_center = np.array([x + np.mean(rot_poly[:,1]), y + np.mean(rot_poly[:,0])])
        arrow_center = box_center + np.array([width * np.cos(theta), width * np.sin(theta)])
        arrow_start = arrow_center - np.array([dx / 2.0, dy / 2.0])       
        ax.arrow(arrow_start[1], arrow_start[0], dy, dx, width=0.02, length_includes_head=True, color='black')
    legend_force = 1000.0
    legend_length = 0.5 * legend_force / zmax
    xs = -scale * 0.85
    ys = scale * 0.85
    dx = legend_length
    dy = 0
    ax.arrow(xs, ys, dx, dy, width=0.02, length_includes_head=True, color='black')
    ax.text(xs + dx * 1.05, ys,  f"{legend_force:.1f} N", color='black')
    return

def plotHexapod(df, ax, time, xmax=800.0, zmax=2000.0, hex="Camera", mult=100):
    xmin = -xmax
    radius = xmax * 0.65
    zmin = -zmax
    ax.set_xlabel("X (um)")
    ax.set_xticks([int(xmin / 2), 0 ,int(xmax / 2)])
    ax.set_ylabel("Y (um)", labelpad=-8)
    ax.set_yticks([int(xmin / 2), 0 ,int(xmax / 2)])
    ax.set_title(f"{hex} Hexapod\n ", fontsize=12, y=1.05)
    ax.set_xlim(xmin, xmax)
    ax.set_ylim(xmin, xmax)
    ax.yaxis.tick_right()
    ax.yaxis.set_label_position("right")
    if len(df) == 0:
        ax.text(xmin * 0.8, 0, "Not Available")
        return
    
    index = df.index.searchsorted(time)
    X = df.iloc[index]["position0"]
    Y = df.iloc[index]["position1"]
    Z = df.iloc[index]["position2"]
    U = df.iloc[index]["position3"]
    V = df.iloc[index]["position4"]
    ax.text(-0.2, 1.05, f'X={X:.1f}um, Y={Y:.1f}um, Z={Z:.1f}um,\nU = {U * 3600.0:.1f}", V = {V * 3600.0:.1f}"',
         transform=ax.transAxes, fontsize=8)
    # X,Y position
    circle = Circle((X,Y), radius, facecolor='yellow',
                    edgecolor='black',linewidth=3)
    ax.add_patch(circle)
    if hex == 'M2': 
        circle2 = Circle((X,Y), radius * 0.5, facecolor='white',
                edgecolor='black',linewidth=3)
        ax.add_patch(circle2)
    ax.scatter(X,Y, marker='+', s=200, color='black')
    ax.scatter(0.0,0.0, marker='+', s=200, color='black')
    # U, V 
    height = xmax / 20.0
    rect = Rectangle((xmin * 0.8, xmax * 0.9), ((xmax - xmin) * 0.7), height, facecolor='black',
                angle=mult * U, rotation_point='center')
    ax.add_patch(rect)
    rect = Rectangle((xmax * 0.9, xmin * 0.8), height, ((xmax - xmin) * 0.7),  facecolor='black',
                angle=mult * V, rotation_point='center')
    ax.add_patch(rect)

    # Z position
    divider = make_axes_locatable(ax)
    ax1 = divider.append_axes("left", size="20%", pad=0.00)
    fig = ax1.get_figure()
    fig.add_axes(ax1)

    height = zmax / 20.0
    rect = Rectangle((0.0, Z - height / 2.0), 1.0, height, facecolor='blue',
                edgecolor='black',linewidth=1)
    ax1.add_patch(rect)
    ax1.set_xlim(0.0,1.0)
    ax1.set_ylim(zmin, zmax)
    ax1.set_xticklabels([])
    if hex == "Camera":
        ax1.set_ylabel("Z (um)", loc='center', labelpad=-15)
    else:
        ax1.set_yticklabels([])
    return

def plotTelescope(az, el, rot, ax, simonyi_file, time):
    # Get the elevation angle at the given time
    ax.set_title("TMA", fontsize=12, y=1.20, clip_on=False)
    ax.set_xlim(0, 1.0)
    ax.set_ylim(0, 1.0)
    ax.set_axis_off()
    if len(el) == 0:
        return
    el_index = el.index.searchsorted(time)
    el_value = el['actualPosition'].iloc[el_index]
    angle = 90.0 - el_value
    tel = plt.imread(M2_simonyi_file)  
    rot_tel = rotate(tel, angle, reshape=True)
    imagebox = OffsetImage(rot_tel, zoom = 0.15)
    ab = AnnotationBbox(imagebox, (0.52, 0.8), frameon = False)
    ax.add_artist(ab)
    ax.text(0.82,0.8, f"Elevation\n{el_value:.2f}", color='black')
    ax.set_axis_off()
    ax.clip_on = False
    #Azimuth
    az_index = az.index.searchsorted(time)
    az_deg = az['actualPosition'].iloc[az_index]%360.0
    az_rad = az_deg * np.pi / 180.0
    radius = 0.2
    az_circle = Circle((0.21, 0.1), radius, facecolor='white', 
                    edgecolor='black',linewidth=3, clip_on=False)
    ax.add_patch(az_circle)
    dx = radius * np.cos(az_rad)
    dy = radius * np.sin(az_rad)
    ax.arrow(0.21, 0.1, dx, dy, width=0.02, length_includes_head=True, color='black', clip_on=False)
    ax.text(0.02, 0.40, f"Azimuth\n{az_deg:.2f}", color='black')
    ax.text(0.42, 0.09, "0", color='black', fontsize=8)
    ax.text(0.17, 0.32, "90", color='black', fontsize=8)
    ax.text(-0.13, 0.06, "180", color='black', fontsize=8)
    ax.text(0.16, -0.17, "270", color='black', fontsize=8)
    #Rotator
    rot_index = rot.index.searchsorted(time)
    rot_deg = rot['actualPosition'].iloc[rot_index]
    rot_rad = rot_deg * np.pi / 180.0
    rot_circle = Circle((0.81, 0.1), radius, facecolor='white', 
                    edgecolor='blue',linewidth=3, clip_on=False)
    ax.add_patch(rot_circle)
    dx = radius * np.cos(rot_rad)
    dy = radius * np.sin(rot_rad)
    ax.arrow(0.81, 0.1, dx, dy, width=0.02, length_includes_head=True, color='blue', clip_on=False)
    ax.text(0.74, 0.40, f"Rotator\n{rot_deg:.2f}", color='blue')
    ax.text(1.03, 0.08, "0", color='blue', fontsize=8)
    ax.text(0.76, 0.32, "90", color='blue', fontsize=8)
    ax.text(0.47, 0.08, "180", color='blue', fontsize=8)
    ax.text(0.76, -0.17, "270", color='blue', fontsize=8)
    return

def addExposures(tel, ax, time):
    time = Time(time, scale='utc').unix_tai
    for i in range(len(tel)):
        if time > Time(tel.index[i], scale='utc').unix_tai \
            and time < Time(tel.iloc[i]['close'], scale='utc').unix_tai:
            seqNum = tel.iloc[i]['imageNumber']
            print(f"Adding image number {seqNum}")
            rect = Rectangle((-0.2, 0.85), 0.4, 0.4, facecolor='lime',
            edgecolor='black',linewidth=2, clip_on=False)
            ax.add_patch(rect)
            ax.text(-0.18, 0.90, f"Exposing\nseqNum\n{seqNum}", fontsize=10)
            return

def dummyAddExposures(tel, ax, time):
    unix_time = Time(time, scale='utc').unix_tai
    tOpen = Time("2024-10-13T19:01:30", scale='utc')
    tClose = Time("2024-10-13T19:02:00", scale='utc')
    if unix_time > tOpen.unix_tai \
        and unix_time < tClose.unix_tai:
        print(f"Adding exposure {time}")
        seqNum = 17
        rect = Rectangle((-0.2, 0.85), 0.4, 0.4, facecolor='lime',
        edgecolor='black',linewidth=2, clip_on=False)
        ax.add_patch(rect)
        ax.text(-0.18, 0.90, f"Exposing\nseqNum\n{seqNum}", fontsize=10)
        return


## Define the times and options

In [ ]:
# Times to start looking at EFD values
# AOS testing
start = Time("2025-08-26T23:30:00", scale='utc')
end = Time("2025-08-27T09:30:00", scale='utc')

M1M3_zmin = -1500
M1M3_zmax = 1500.0
M1M3_lateralMax = 2000.0

M2_yaml_file = '../../MTM2/cell_geom/cell_geom.yaml'
M2_simonyi_file = '../../MTM2/cell_geom/Simonyi.png'

M2_axial_zmax = 500
M2_tangent_zmax = 2000

camXmax = 500
camZmax = 2000
m2Xmax = 8
m2Zmax = 10

timestep = 5.0 # step in seconds 

# This speeds up the M1M3 queries by only getting the necessary items
# About 2X faster with this.
M1M3_AOS_names = []
for i in range(len(FATable)):
    name=f"zForces{i}"
    M1M3_AOS_names.append(name)
for i in range(len(FATable)):
    if FATable[i].actuator_type.name == 'DAA':
        if FATable[i].orientation.name in ['X_PLUS', 'X_MINUS']:
            name = f"xForces{FATable[i].x_index}"
            M1M3_AOS_names.append(name)
        if FATable[i].orientation.name in ['Y_PLUS', 'Y_MINUS']:
            name = f"yForces{FATable[i].y_index}"
            M1M3_AOS_names.append(name)

M1M3_names = []
for i in range(len(FATable)):
    name=f"zForce{i}"
    M1M3_names.append(name)
for i in range(len(FATable)):
    if FATable[i].actuator_type.name == 'DAA':
        if FATable[i].orientation.name in ['X_PLUS', 'X_MINUS']:
            name = f"xForce{FATable[i].x_index}"
            M1M3_names.append(name)
        if FATable[i].orientation.name in ['Y_PLUS', 'Y_MINUS']:
            name = f"yForce{FATable[i].y_index}"
            M1M3_names.append(name)


M2_AOS_names = []
M2_meas_names = []
for i in range(72):
    name=f"axial{i}"
    M2_AOS_names.append(name)
    name=f"measured{i}"
    M2_meas_names.append(name)


## Now generate the frames
### This will take some time
### First, pull the data from the EFD

In [ ]:
client = EfdClient('usdf_efd')
timestamp = start.isot.split('.')[0].replace('-','').replace(':','')
dirName = f"/home/c/cslage/u/MTAOS/movies/movie_{timestamp}"
%mkdir -p {dirName}
movieName = f"Simonyi_movie_{timestamp}.mp4"
print(movieName)

async def trimmedDataframe(t0, timestep, topic, fields, start, end, salindex=None, time_jump = 300):
    # This only keeps the dataframe with rows as specified by the 'timestep' value
    # This reduces the size of the dataframes and speeds things up.
    this_start = start
    this_end = this_start
    trimmed_df = []
    time = t0
    print(topic)
    counter = 1
    while this_end < pd.Timestamp(end.isot, tz='utc'):
        if counter % 10 == 0:
            print(topic, this_start.isot)
        this_end = this_start + pd.Timedelta(seconds=time_jump)
        if not salindex:
            df = await client.select_time_series(topic, fields, this_start, this_end)
        else:
            df = await client.select_time_series(topic, fields, this_start, this_end, index=salindex)
        df_length = len(df)
        index_list = []
        while time < pd.Timestamp(this_end.isot, tz='utc'):
            index = df.index.searchsorted(time)
            if index > df_length - 1:
                break
            index_list.append(index)
            time += pd.Timedelta(seconds=timestep)
        this_df = df.iloc[index_list]
        if len(trimmed_df) == 0:
            trimmed_df = this_df
        else:
            trimmed_df = pd.concat([trimmed_df, this_df])
        this_start = this_start + pd.Timedelta(seconds=time_jump)
        counter += 1
    return trimmed_df

t88 = Time.now()
#Get the data from the EFD
t0 = pd.Timestamp(start.isot, tz='utc')

el = await trimmedDataframe(t0, timestep, "lsst.sal.MTMount.elevation", 
                                 ['actualPosition'], start, end)
print(len(el))

az = await trimmedDataframe(t0, timestep, "lsst.sal.MTMount.azimuth", 
                                 ['actualPosition'], start, end)
print(len(az))

rot = await trimmedDataframe(t0, timestep, "lsst.sal.MTRotator.rotation", 
                                 ['actualPosition'], start, end)
print(len(rot))

M1M3_forces = await trimmedDataframe(t0, timestep, "lsst.sal.MTM1M3.forceActuatorData", 
                                 M1M3_names, start, end)
print(len(M1M3_forces))

print("lsst.sal.MTM1M3.command_applyActiveOpticForces")
M1M3_AOS = await client.select_time_series("lsst.sal.MTM1M3.command_applyActiveOpticForces", 
                                 M1M3_AOS_names, start, end)
print(len(M1M3_AOS))

M2_axial_forces = await trimmedDataframe(t0, timestep, "lsst.sal.MTM2.axialForce", 
                                 M2_meas_names, start, end)
print(len(M2_axial_forces))

print("lsst.sal.MTM2.command_applyForces")
M2_AOS = await client.select_time_series("lsst.sal.MTM2.command_applyForces", 
                                 M2_AOS_names, start, end)
print(len(M2_AOS))

M2_tangent_forces = await trimmedDataframe(t0, timestep, "lsst.sal.MTM2.tangentForce", 
                                 ["*"], start, end)
print(len(M2_tangent_forces))

camhex = await trimmedDataframe(t0, timestep, "lsst.sal.MTHexapod.application", 
                                 ["*"], start, end, salindex=1)

print(len(camhex))
m2hex = await trimmedDataframe(t0, timestep, "lsst.sal.MTHexapod.application", 
                                 ["*"], start, end, salindex=2)
print(len(m2hex))

print("lsst.sal.MTCamera.logevent_endOfImageTelemetry")
tel = await client.select_time_series("lsst.sal.MTCamera.logevent_endOfImageTelemetry", \
                                            ['imageNumber', 'timestampAcquisitionStart', 'exposureTime'], \
                                             start, end)
if len(tel) > 0:
    tel['close'] = tel['timestampAcquisitionStart'] + tel['exposureTime']
    tel['close'] = Time(tel['close'], format='unix_tai', scale='tai').utc.isot
    tel['timestampAcquisitionStart'] = Time(tel['timestampAcquisitionStart'], format='unix_tai', scale='tai').utc.isot
    tel = tel.set_index('timestampAcquisitionStart')
print(len(tel))
t89 = Time.now()
print(f"Took {((t89 - t88).value * 86400):.1f} seconds")

In [ ]:
tel = await client.select_time_series("lsst.sal.MTCamera.logevent_endOfImageTelemetry", \
                                            ['imageNumber', 'timestampAcquisitionStart', 'exposureTime'], \
                                             start, end)
if len(tel) > 0:
    tel['close'] = tel['timestampAcquisitionStart'] + tel['exposureTime']
    tel['close'] = Time(tel['close'], format='unix_tai', scale='tai').utc.isot
    tel['timestampAcquisitionStart'] = Time(tel['timestampAcquisitionStart'], format='unix_tai', scale='tai').utc.isot
    tel = tel.set_index('timestampAcquisitionStart')
print(len(tel))


## Adjust the limits

In [ ]:
def roundUp(value):
    # Round up the value to something even
    if value < 1E-6:
        return value
    mag = int(np.log10(value))
    rounded_value = (int(value / 10 ** mag) + 1) * 10 ** mag
    return rounded_value

if len(M1M3_AOS.values) == 0:
    m1m3_aos_max = 10.0
else:
    m1m3_aos_max = max(np.nanmax(np.array(M1M3_AOS.values, dtype=float)), \
                          -np.nanmin(np.array(M1M3_AOS.values, dtype=float)))
    m1m3_aos_max = roundUp(m1m3_aos_max)
if len(M2_AOS.values) == 0:
    m2_aos_max = 10.0
else:
    m2_aos_max = max(np.nanmax(np.array(M2_AOS.values, dtype=float)), \
                     -np.nanmin(np.array(M2_AOS.values, dtype=float)))   
    m2_aos_max = roundUp(m2_aos_max)

minLimitX = 7.0 # Always have the hexapod limit be at least +/- 8 microns
minLimitZ = 9.0 # Always have the hexapod limit be at least +/- 10 microns
camXmax = abs(np.max([np.max(camhex['position0'].values), -np.min(camhex['position0'].values),
                np.max(camhex['position1'].values), -np.min(camhex['position1'].values), 
                minLimitX]))
camXmax = roundUp(camXmax)
camZmax = abs(np.max([np.max(camhex['position2'].values), -np.min(camhex['position2'].values), minLimitZ]))
camZmax = roundUp(camZmax)

m2Xmax = abs(np.max([np.max(m2hex['position0'].values), -np.min(m2hex['position0'].values),
                np.max(m2hex['position1'].values), -np.min(m2hex['position1'].values), 
                minLimitX]))
m2Xmax = roundUp(m2Xmax)
m2Zmax = abs(np.max([np.max(m2hex['position2'].values), -np.min(m2hex['position2'].values), minLimitZ]))
m2Zmax = roundUp(m2Zmax)
print(camXmax, camZmax, m2Xmax, m2Zmax)
print(m1m3_aos_max, m2_aos_max)

### Now build the frames

In [ ]:
# Build the individual frames
fig = plt.figure(figsize=(10, 5.7))

frame = 0
time = t0
while time < pd.Timestamp(end.isot, tz='utc'):
    t98 = Time.now()
    axs = []
    axs.append(plt.axes([0.08,0.56,0.20,0.35]))
    axs.append(plt.axes([0.28,0.56,0.20,0.35]))
    axs.append(plt.axes([0.48,0.56,0.20,0.35]))
    axs.append(plt.axes([0.06,0.07,0.20,0.35]))
    axs.append(plt.axes([0.26,0.07,0.20,0.35]))
    axs.append(plt.axes([0.53,0.09,0.20,0.30]))
    axs.append(plt.axes([0.73,0.09,0.20,0.30]))
    axs.append(plt.axes([0.77,0.56,0.17,0.29]))

    lateralForcesM1M3(M1M3_forces, axs[0], FATable, time, M1M3_lateralMax)
    heatMapZM1M3(M1M3_AOS, axs[1], FATable, time, -m1m3_aos_max, m1m3_aos_max, AOS=True)
    heatMapZM1M3(M1M3_forces, axs[2], FATable, time, M1M3_zmin, M1M3_zmax, AOS=False)
    plotAxialForcesM2(M2_AOS, M2_yaml_file, axs[3], -m2_aos_max, m2_aos_max, time, AOS=True)
    #plotTangentForcesM2(M2_tangent_forces, M2_yaml_file, axs[3], M2_tangent_zmax, time)
    plotAxialForcesM2(M2_axial_forces, M2_yaml_file, axs[4], -M2_axial_zmax, M2_axial_zmax, time, AOS=False)
    plotTangentForcesM2(M2_tangent_forces, M2_yaml_file, axs[4], M2_tangent_zmax, time)
    plotHexapod(camhex, axs[5], time, xmax=camXmax, zmax=camZmax, hex="Camera") 
    plotHexapod(m2hex, axs[6], time, xmax=m2Xmax, zmax=m2Zmax, hex="M2")
    plotTelescope(az, el, rot, axs[7], M2_simonyi_file, time)
    addExposures(tel, axs[7], time)
    #dummyAddExposures(tel, axs[7], time)    
    plt.suptitle(f"Simonyi telescope movie {time}", fontsize=12)
    plt.savefig(f"{dirName}/Frame_{frame:05d}.png")
    if frame % 10 == 0:
        print(f"Finished frame {frame}")
    frame += 1
    time += pd.Timedelta(seconds=timestep)
    plt.clf()
    t99 = Time.now()
    #print(f"Frame {frame} took {((t99 - t98).value * 86400 * 1000):.1f} msec.")
    #if frame > 10:
    #    break


In [ ]:
print(f"\033[1mThe movie name will be: {dirName}/{movieName}\033[0m")

command = f"ffmpeg -pattern_type glob -i '{dirName}/*.png' -f mp4 -vcodec libx264 -pix_fmt yuv420p -framerate 50 -y {dirName}/{movieName}"
args = shlex.split(command)
build_movie = subprocess.Popen(args)
build_movie.wait()